In [1]:
import pandas as pd
import numpy as np
import os

from typing import List, Tuple

import catboost as cb
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, KFold

import scipy.stats as st
from scipy.stats import probplot, ks_2samp
import missingno as msno

%matplotlib inline
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(color_codes=True)
sns.set()
sns.set_palette('husl')


pd.options.display.max_rows=100
pd.options.display.max_columns=250

import warnings
warnings.filterwarnings("ignore")

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'ATaganova_submission.csv',
 'catboost_info',
 'EDA-Copy2.ipynb',
 'EDA-Copy3.ipynb',
 'EDA.ipynb',
 'EDA_ProfileReport.ipynb',
 'geekbrains-competitive-data-analysis.zip',
 'hw_4.ipynb',
 'row_data',
 'webinar4_baseline.ipynb']

In [3]:
path = 'row_data'
os.listdir(path)

['applications_history.csv',
 'bki.csv',
 'client_profile.csv',
 'payments.csv',
 'test.csv',
 'train.csv']

In [4]:
train_df = pd.read_csv(f'{path}/train.csv')
test_df = pd.read_csv(f'{path}/test.csv')

train_df.shape, test_df.shape

((110093, 3), (165141, 2))

In [5]:
# bki_df = pd.read_csv(f'{path}/bki.csv')
# bki_df.describe()

In [6]:
# bki_df_feats = set(bki_df.columns.tolist())
# bki_df_feats

In [7]:
client_profile_df = pd.read_csv(f'{path}/client_profile.csv')
client_profile_df.describe()

,APPLICATION_NUMBER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,FLAG_EMAIL,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,2.500000e+05,250000.000000,2.500000e+05,2.500000e+05,249989.000000,250000.000000,250000.000000,250000.000000,85041.000000,250000.000000,250000.000000,249998.000000,109121.000000,2.494680e+05,200462.000000,216293.000000,216293.000000,216293.000000,216293.000000,216293.000000,216293.000000
mean,1.235771e+08,0.417624,1.690662e+05,5.993051e+05,27125.209585,0.020871,16033.460496,67505.883296,12.030221,0.280244,0.057072,2.154025,0.502122,5.141749e-01,0.510848,0.006348,0.006866,0.034217,0.267475,0.266786,1.899030
std,8.877722e+04,0.722529,2.588480e+05,4.022720e+05,14503.930366,0.013827,4361.332320,139261.099609,11.899404,0.449119,0.231981,0.911543,0.210817,1.910938e-01,0.195054,0.083783,0.107957,0.204029,0.914514,0.831547,1.868811
min,1.234233e+08,0.000000,2.565000e+04,4.500000e+04,1615.500000,0.000290,7489.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.014691,8.173617e-08,0.000527,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.235002e+08,0.000000,1.125000e+05,2.700000e+05,16524.000000,0.010006,12415.000000,931.000000,5.000000,0.000000,0.000000,2.000000,0.334180,3.920026e-01,0.370650,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.235772e+08,0.000000,1.485000e+05,5.147775e+05,24930.000000,0.018850,15744.000000,2217.000000,9.000000,0.000000,0.000000,2.000000,0.506018,5.656713e-01,0.535276,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.236540e+08,1.000000,2.025000e+05,8.086500e+05,34618.500000,0.028663,19681.000000,5678.000000,15.000000,1.000000,0.000000,3.000000,0.674793,6.635195e-01,0.669057,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,1.237309e+08,19.000000,1.170000e+08,4.050000e+06,230161.500000,0.072508,25229.000000,365243.000000,69.000000,1.000000,1.000000,20.000000,0.962693,8.549997e-01,0.896010,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


In [8]:
# client_profile_df_feats = set(client_profile_df.columns.tolist())
# client_profile_df_feats

In [9]:
# payments_df = pd.read_csv(f'{path}/payments.csv')
# payments_df.describe()

In [10]:
# payments_df_feats = set(payments_df.columns.tolist())
# payments_df_feats

In [11]:
# applications_history_df = pd.read_csv(f'{path}/applications_history.csv')
# applications_history_df.describe()

In [12]:
# applications_history_df_feats = set(applications_history_df.columns.tolist())
# applications_history_df_feats

In [13]:
# applications_history_df.info()

In [14]:
# applications_history_df['APPLICATION_NUMBER'].nunique()

In [15]:
# payments_df.info()

In [16]:
# payments_df['APPLICATION_NUMBER'].nunique()

In [17]:
train_client_profile = train_df.merge(client_profile_df, how='left')
train_client_profile.head(2)

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,FLAG_EMAIL,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,123687442,0,Cash,M,1.0,157500.0,855000.0,25128.0,Secondary / secondary special,Married,0.019101,15728.0,1719.0,11.0,0.0,0.0,3.0,0.700784,0.645914,0.71657,0.0,0.0,1.0,0.0,0.0,2.0
1,123597908,1,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
test_client_profile = test_df.merge(client_profile_df, how='left')
test_client_profile.head(2)

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,FLAG_EMAIL,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,123724268,Cash,M,0.0,117000.0,1125000.0,32895.0,Secondary / secondary special,Married,0.028663,16007.0,2646.0,20.0,0.0,0.0,2.0,NaN,0.628266,NaN,0.0,0.0,0.0,0.0,1.0,4.0
1,123456549,Cash,F,2.0,81000.0,312768.0,17095.5,Secondary / secondary special,Married,0.019689,10315.0,459.0,NaN,0.0,0.0,4.0,NaN,0.578161,0.18849,0.0,0.0,1.0,0.0,0.0,2.0


## Задание 1

#### Обучить алгоритмы LightGBM и XGBoost, получить OOF прогнозы, оценить корреляцию прогнозов на обучающей выборке. Применить модели на тестовую выборку и оценить корреляцию.

Подготовка данных

In [31]:
num_feats = train_client_profile.select_dtypes('number').drop(columns=['APPLICATION_NUMBER','TARGET']).columns.tolist()
cat_feats = train_client_profile.select_dtypes('object').columns.tolist()

X = train_client_profile[num_feats+cat_feats]

y = train_client_profile['TARGET']

In [20]:
import time
def xgb_cross_validation(params, X, y, cv):
    """
    Кросс-валидация для модели XGBoost.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признако для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    cv: KFold or StratifiedKFold generator.
        Объект KFold / StratifiedKFold для определения
        стратегии кросс-валидации модели.

    categorical: str, optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    estimators: list
        Список с объектами обученной модели.

    oof_preds: np.array
        Вектор OOF-прогнозов.

    """
    estimators, folds_scores = [], []
    oof_preds = np.zeros(X.shape[0])

    print(f"{time.ctime()}, Cross-Validation, {X.shape[0]} rows, {X.shape[1]} cols")
    

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):

        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model = xgb.XGBClassifier(**params)
        model.fit(
            x_train, y_train,
            eval_set=[(x_train, y_train), (x_valid, y_valid)]
        )
        oof_preds[valid_idx] = model.predict_proba(x_valid)[:, 1]
        score = roc_auc_score(y_valid, oof_preds[valid_idx])
        print(f"Fold {fold+1}, Valid score = {round(score, 5)}")
        folds_scores.append(round(score, 5))
        estimators.append(model)

    print(f"Score by each fold: {folds_scores}")
    print("="*65)
    return estimators, oof_preds


In [21]:
cb_params = {
    "n_estimators": 1000,
    "learning_rate": 0.01,
    "eval_metric": "auc",
    "max_bin": 20,
    "max_depth": 6,
    "random_seed": 42
}

cv = KFold(n_splits=5, random_state=1234123, shuffle=True)
X_dummies = pd.get_dummies(X)

estimators_xgb, oof_preds_xgb = xgb_cross_validation(
    params=cb_params, X=X_dummies, y=y, cv=cv
)

Tue Sep 29 15:33:53 2020, Cross-Validation, 110093 rows, 36 cols
[15:33:54] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { random_seed } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-auc:0.70053	validation_1-auc:0.68739
[1]	validation_0-auc:0.70513	validation_1-auc:0.69024
[2]	validation_0-auc:0.70504	validation_1-auc:0.69021
[3]	validation_0-auc:0.70526	validation_1-auc:0.69044
[4]	validation_0-auc:0.70536	validation_1-auc:0.69051
[5]	validation_0-auc:0.70571	validation_1-auc:0.69067
[6]	validation_0-auc:0.70682	validation_1-auc:0.69208
[7]	validation_0-auc:0.70670	validation_1-auc:0.69197
[8]	validation_0-auc:0.70745	validation_1-auc:0.69320
[9]	validation_0-auc:0.70773	validation_1-auc:0.69284
[10]	v

[141]	validation_0-auc:0.73137	validation_1-auc:0.70289
[142]	validation_0-auc:0.73147	validation_1-auc:0.70295
[143]	validation_0-auc:0.73156	validation_1-auc:0.70297
[144]	validation_0-auc:0.73162	validation_1-auc:0.70296
[145]	validation_0-auc:0.73168	validation_1-auc:0.70299
[146]	validation_0-auc:0.73181	validation_1-auc:0.70291
[147]	validation_0-auc:0.73184	validation_1-auc:0.70283
[148]	validation_0-auc:0.73200	validation_1-auc:0.70285
[149]	validation_0-auc:0.73208	validation_1-auc:0.70302
[150]	validation_0-auc:0.73223	validation_1-auc:0.70299
[151]	validation_0-auc:0.73231	validation_1-auc:0.70293
[152]	validation_0-auc:0.73233	validation_1-auc:0.70306
[153]	validation_0-auc:0.73240	validation_1-auc:0.70317
[154]	validation_0-auc:0.73257	validation_1-auc:0.70327
[155]	validation_0-auc:0.73266	validation_1-auc:0.70322
[156]	validation_0-auc:0.73271	validation_1-auc:0.70319
[157]	validation_0-auc:0.73291	validation_1-auc:0.70317
[158]	validation_0-auc:0.73302	validation_1-auc:

[288]	validation_0-auc:0.74867	validation_1-auc:0.70766
[289]	validation_0-auc:0.74868	validation_1-auc:0.70774
[290]	validation_0-auc:0.74884	validation_1-auc:0.70767
[291]	validation_0-auc:0.74897	validation_1-auc:0.70757
[292]	validation_0-auc:0.74907	validation_1-auc:0.70778
[293]	validation_0-auc:0.74929	validation_1-auc:0.70763
[294]	validation_0-auc:0.74948	validation_1-auc:0.70766
[295]	validation_0-auc:0.74955	validation_1-auc:0.70770
[296]	validation_0-auc:0.74968	validation_1-auc:0.70784
[297]	validation_0-auc:0.74988	validation_1-auc:0.70800
[298]	validation_0-auc:0.75012	validation_1-auc:0.70793
[299]	validation_0-auc:0.75033	validation_1-auc:0.70792
[300]	validation_0-auc:0.75043	validation_1-auc:0.70806
[301]	validation_0-auc:0.75051	validation_1-auc:0.70822
[302]	validation_0-auc:0.75071	validation_1-auc:0.70819
[303]	validation_0-auc:0.75090	validation_1-auc:0.70817
[304]	validation_0-auc:0.75111	validation_1-auc:0.70819
[305]	validation_0-auc:0.75123	validation_1-auc:

[435]	validation_0-auc:0.76658	validation_1-auc:0.71361
[436]	validation_0-auc:0.76671	validation_1-auc:0.71352
[437]	validation_0-auc:0.76682	validation_1-auc:0.71351
[438]	validation_0-auc:0.76689	validation_1-auc:0.71353
[439]	validation_0-auc:0.76699	validation_1-auc:0.71366
[440]	validation_0-auc:0.76702	validation_1-auc:0.71382
[441]	validation_0-auc:0.76712	validation_1-auc:0.71390
[442]	validation_0-auc:0.76727	validation_1-auc:0.71386
[443]	validation_0-auc:0.76732	validation_1-auc:0.71390
[444]	validation_0-auc:0.76745	validation_1-auc:0.71374
[445]	validation_0-auc:0.76759	validation_1-auc:0.71385
[446]	validation_0-auc:0.76769	validation_1-auc:0.71381
[447]	validation_0-auc:0.76772	validation_1-auc:0.71391
[448]	validation_0-auc:0.76780	validation_1-auc:0.71386
[449]	validation_0-auc:0.76790	validation_1-auc:0.71396
[450]	validation_0-auc:0.76793	validation_1-auc:0.71388
[451]	validation_0-auc:0.76798	validation_1-auc:0.71390
[452]	validation_0-auc:0.76809	validation_1-auc:

[582]	validation_0-auc:0.77971	validation_1-auc:0.71558
[583]	validation_0-auc:0.77979	validation_1-auc:0.71553
[584]	validation_0-auc:0.77995	validation_1-auc:0.71542
[585]	validation_0-auc:0.78004	validation_1-auc:0.71548
[586]	validation_0-auc:0.78005	validation_1-auc:0.71546
[587]	validation_0-auc:0.78012	validation_1-auc:0.71552
[588]	validation_0-auc:0.78025	validation_1-auc:0.71538
[589]	validation_0-auc:0.78044	validation_1-auc:0.71539
[590]	validation_0-auc:0.78056	validation_1-auc:0.71558
[591]	validation_0-auc:0.78060	validation_1-auc:0.71558
[592]	validation_0-auc:0.78081	validation_1-auc:0.71556
[593]	validation_0-auc:0.78095	validation_1-auc:0.71564
[594]	validation_0-auc:0.78107	validation_1-auc:0.71569
[595]	validation_0-auc:0.78111	validation_1-auc:0.71563
[596]	validation_0-auc:0.78116	validation_1-auc:0.71555
[597]	validation_0-auc:0.78126	validation_1-auc:0.71556
[598]	validation_0-auc:0.78131	validation_1-auc:0.71559
[599]	validation_0-auc:0.78145	validation_1-auc:

...

[380]	validation_0-auc:0.75784	validation_1-auc:0.71481
[381]	validation_0-auc:0.75799	validation_1-auc:0.71486
[382]	validation_0-auc:0.75816	validation_1-auc:0.71483
[383]	validation_0-auc:0.75815	validation_1-auc:0.71488
[384]	validation_0-auc:0.75824	validation_1-auc:0.71508
[385]	validation_0-auc:0.75839	validation_1-auc:0.71501
[386]	validation_0-auc:0.75858	validation_1-auc:0.71504
[387]	validation_0-auc:0.75875	validation_1-auc:0.71510
[388]	validation_0-auc:0.75883	validation_1-auc:0.71508
[389]	validation_0-auc:0.75893	validation_1-auc:0.71507
[390]	validation_0-auc:0.75901	validation_1-auc:0.71516
[391]	validation_0-auc:0.75912	validation_1-auc:0.71520
[392]	validation_0-auc:0.75916	validation_1-auc:0.71527
[393]	validation_0-auc:0.75933	validation_1-auc:0.71544
[394]	validation_0-auc:0.75942	validation_1-auc:0.71538
[395]	validation_0-auc:0.75954	validation_1-auc:0.71528
[396]	validation_0-auc:0.75960	validation_1-auc:0.71529
[397]	validation_0-auc:0.75977	validation_1-auc:

[527]	validation_0-auc:0.77173	validation_1-auc:0.71881
[528]	validation_0-auc:0.77185	validation_1-auc:0.71888
[529]	validation_0-auc:0.77194	validation_1-auc:0.71881
[530]	validation_0-auc:0.77200	validation_1-auc:0.71883
[531]	validation_0-auc:0.77207	validation_1-auc:0.71885
[532]	validation_0-auc:0.77221	validation_1-auc:0.71872
[533]	validation_0-auc:0.77228	validation_1-auc:0.71892
[534]	validation_0-auc:0.77232	validation_1-auc:0.71897
[535]	validation_0-auc:0.77246	validation_1-auc:0.71903
[536]	validation_0-auc:0.77252	validation_1-auc:0.71898
[537]	validation_0-auc:0.77256	validation_1-auc:0.71898
[538]	validation_0-auc:0.77267	validation_1-auc:0.71901
[539]	validation_0-auc:0.77278	validation_1-auc:0.71905
[540]	validation_0-auc:0.77284	validation_1-auc:0.71905
[541]	validation_0-auc:0.77287	validation_1-auc:0.71906
[542]	validation_0-auc:0.77300	validation_1-auc:0.71901
[543]	validation_0-auc:0.77309	validation_1-auc:0.71894
[544]	validation_0-auc:0.77316	validation_1-auc:

[674]	validation_0-auc:0.78197	validation_1-auc:0.71974
[675]	validation_0-auc:0.78202	validation_1-auc:0.71972
[676]	validation_0-auc:0.78204	validation_1-auc:0.71971
[677]	validation_0-auc:0.78210	validation_1-auc:0.71973
[678]	validation_0-auc:0.78212	validation_1-auc:0.71972
[679]	validation_0-auc:0.78216	validation_1-auc:0.71972
[680]	validation_0-auc:0.78219	validation_1-auc:0.71972
[681]	validation_0-auc:0.78226	validation_1-auc:0.71973
[682]	validation_0-auc:0.78231	validation_1-auc:0.71978
[683]	validation_0-auc:0.78237	validation_1-auc:0.71977
[684]	validation_0-auc:0.78246	validation_1-auc:0.71975
[685]	validation_0-auc:0.78249	validation_1-auc:0.71965
[686]	validation_0-auc:0.78251	validation_1-auc:0.71962
[687]	validation_0-auc:0.78250	validation_1-auc:0.71962
[688]	validation_0-auc:0.78259	validation_1-auc:0.71972
[689]	validation_0-auc:0.78270	validation_1-auc:0.71966
[690]	validation_0-auc:0.78280	validation_1-auc:0.71957
[691]	validation_0-auc:0.78282	validation_1-auc:

[821]	validation_0-auc:0.78899	validation_1-auc:0.72097
[822]	validation_0-auc:0.78901	validation_1-auc:0.72101
[823]	validation_0-auc:0.78901	validation_1-auc:0.72101
[824]	validation_0-auc:0.78900	validation_1-auc:0.72107
[825]	validation_0-auc:0.78909	validation_1-auc:0.72098
[826]	validation_0-auc:0.78911	validation_1-auc:0.72097
[827]	validation_0-auc:0.78914	validation_1-auc:0.72098
[828]	validation_0-auc:0.78919	validation_1-auc:0.72095
[829]	validation_0-auc:0.78922	validation_1-auc:0.72095
[830]	validation_0-auc:0.78935	validation_1-auc:0.72098
[831]	validation_0-auc:0.78933	validation_1-auc:0.72100
[832]	validation_0-auc:0.78934	validation_1-auc:0.72109
[833]	validation_0-auc:0.78941	validation_1-auc:0.72119
[834]	validation_0-auc:0.78943	validation_1-auc:0.72120
[835]	validation_0-auc:0.78948	validation_1-auc:0.72113
[836]	validation_0-auc:0.78956	validation_1-auc:0.72113
[837]	validation_0-auc:0.78960	validation_1-auc:0.72112
[838]	validation_0-auc:0.78964	validation_1-auc:

[968]	validation_0-auc:0.79449	validation_1-auc:0.72158
[969]	validation_0-auc:0.79450	validation_1-auc:0.72158
[970]	validation_0-auc:0.79452	validation_1-auc:0.72158
[971]	validation_0-auc:0.79453	validation_1-auc:0.72157
[972]	validation_0-auc:0.79455	validation_1-auc:0.72156
[973]	validation_0-auc:0.79458	validation_1-auc:0.72156
[974]	validation_0-auc:0.79461	validation_1-auc:0.72156
[975]	validation_0-auc:0.79471	validation_1-auc:0.72149
[976]	validation_0-auc:0.79473	validation_1-auc:0.72149
[977]	validation_0-auc:0.79475	validation_1-auc:0.72151
[978]	validation_0-auc:0.79476	validation_1-auc:0.72142
[979]	validation_0-auc:0.79480	validation_1-auc:0.72145
[980]	validation_0-auc:0.79482	validation_1-auc:0.72144
[981]	validation_0-auc:0.79483	validation_1-auc:0.72144
[982]	validation_0-auc:0.79485	validation_1-auc:0.72144
[983]	validation_0-auc:0.79485	validation_1-auc:0.72144
[984]	validation_0-auc:0.79492	validation_1-auc:0.72142
[985]	validation_0-auc:0.79501	validation_1-auc:

In [22]:
# XGBoost
# Score by each fold: [0.716, 0.71983, 0.72307, 0.71301, 0.72161]

In [43]:
def light_cross_validation(X, y, cv):
    """
    Кросс-валидация для модели LightGBM.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признако для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    cv: KFold or StratifiedKFold generator.
        Объект KFold / StratifiedKFold для определения
        стратегии кросс-валидации модели.

    categorical: str, optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    estimators: list
        Список с объектами обученной модели.

    oof_preds: np.array
        Вектор OOF-прогнозов.

    """
    estimators, folds_scores = [], []
    oof_preds = np.zeros(X.shape[0])

    print(f"{time.ctime()}, Cross-Validation, {X.shape[0]} rows, {X.shape[1]} cols")

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):

        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model = lgb.LGBMClassifier(n_estimators=1000,  learning_rate=0.01, random_state=1, n_jobs=-1, silent=True)
        model.fit(
            x_train, y_train, 
            eval_set=[(x_train, y_train), (x_valid, y_valid)]
        )
        oof_preds[valid_idx] = model.predict_proba(x_valid)[:, 1]
        score = roc_auc_score(y_valid, oof_preds[valid_idx])
        print(f"Fold {fold+1}, Valid score = {round(score, 5)}")
        folds_scores.append(round(score, 5))
        estimators.append(model)

    print(f"Score by each fold: {folds_scores}")
    print("="*65)
    return estimators, oof_preds

In [45]:
cv = KFold(n_splits=5, random_state=1234123, shuffle=True)

estimators_light, oof_preds_light = light_cross_validation(
    X=X[num_feats], y=y, cv=cv,
)

Tue Sep 29 16:02:09 2020, Cross-Validation, 110093 rows, 20 cols
[1]	training's binary_logloss: 0.278297	valid_1's binary_logloss: 0.287987
[2]	training's binary_logloss: 0.277789	valid_1's binary_logloss: 0.287522
[3]	training's binary_logloss: 0.277295	valid_1's binary_logloss: 0.287075
[4]	training's binary_logloss: 0.276821	valid_1's binary_logloss: 0.286632
[5]	training's binary_logloss: 0.276359	valid_1's binary_logloss: 0.286216
[6]	training's binary_logloss: 0.275908	valid_1's binary_logloss: 0.285809
[7]	training's binary_logloss: 0.275473	valid_1's binary_logloss: 0.285411
[8]	training's binary_logloss: 0.275052	valid_1's binary_logloss: 0.285025
[9]	training's binary_logloss: 0.274646	valid_1's binary_logloss: 0.28466
[10]	training's binary_logloss: 0.274249	valid_1's binary_logloss: 0.284289
[11]	training's binary_logloss: 0.273864	valid_1's binary_logloss: 0.283939
[12]	training's binary_logloss: 0.273487	valid_1's binary_logloss: 0.283594
[13]	training's binary_logloss: 0

[108]	training's binary_logloss: 0.256686	valid_1's binary_logloss: 0.269363
[109]	training's binary_logloss: 0.2566	valid_1's binary_logloss: 0.269301
[110]	training's binary_logloss: 0.256516	valid_1's binary_logloss: 0.26924
[111]	training's binary_logloss: 0.256436	valid_1's binary_logloss: 0.269181
[112]	training's binary_logloss: 0.256346	valid_1's binary_logloss: 0.269137
[113]	training's binary_logloss: 0.256259	valid_1's binary_logloss: 0.269083
[114]	training's binary_logloss: 0.256177	valid_1's binary_logloss: 0.269029
[115]	training's binary_logloss: 0.256095	valid_1's binary_logloss: 0.268969
[116]	training's binary_logloss: 0.256011	valid_1's binary_logloss: 0.268919
[117]	training's binary_logloss: 0.255927	valid_1's binary_logloss: 0.268881
[118]	training's binary_logloss: 0.255843	valid_1's binary_logloss: 0.268825
[119]	training's binary_logloss: 0.255764	valid_1's binary_logloss: 0.268781
[120]	training's binary_logloss: 0.255687	valid_1's binary_logloss: 0.26873
[12

[214]	training's binary_logloss: 0.250123	valid_1's binary_logloss: 0.265989
[215]	training's binary_logloss: 0.250077	valid_1's binary_logloss: 0.265983
[216]	training's binary_logloss: 0.250027	valid_1's binary_logloss: 0.265958
[217]	training's binary_logloss: 0.249982	valid_1's binary_logloss: 0.265945
[218]	training's binary_logloss: 0.249938	valid_1's binary_logloss: 0.265927
[219]	training's binary_logloss: 0.249897	valid_1's binary_logloss: 0.265918
[220]	training's binary_logloss: 0.249851	valid_1's binary_logloss: 0.265914
[221]	training's binary_logloss: 0.249804	valid_1's binary_logloss: 0.265904
[222]	training's binary_logloss: 0.249756	valid_1's binary_logloss: 0.26588
[223]	training's binary_logloss: 0.249717	valid_1's binary_logloss: 0.265869
[224]	training's binary_logloss: 0.249669	valid_1's binary_logloss: 0.265856
[225]	training's binary_logloss: 0.249628	valid_1's binary_logloss: 0.265849
[226]	training's binary_logloss: 0.249589	valid_1's binary_logloss: 0.265841


[334]	training's binary_logloss: 0.24559	valid_1's binary_logloss: 0.265021
[335]	training's binary_logloss: 0.24556	valid_1's binary_logloss: 0.26502
[336]	training's binary_logloss: 0.245529	valid_1's binary_logloss: 0.265018
[337]	training's binary_logloss: 0.245499	valid_1's binary_logloss: 0.265013
[338]	training's binary_logloss: 0.24547	valid_1's binary_logloss: 0.265005
[339]	training's binary_logloss: 0.245438	valid_1's binary_logloss: 0.264999
[340]	training's binary_logloss: 0.245408	valid_1's binary_logloss: 0.264999
[341]	training's binary_logloss: 0.245378	valid_1's binary_logloss: 0.264996
[342]	training's binary_logloss: 0.245346	valid_1's binary_logloss: 0.264988
[343]	training's binary_logloss: 0.245315	valid_1's binary_logloss: 0.264984
[344]	training's binary_logloss: 0.245283	valid_1's binary_logloss: 0.264981
[345]	training's binary_logloss: 0.245254	valid_1's binary_logloss: 0.264977
[346]	training's binary_logloss: 0.245228	valid_1's binary_logloss: 0.264976
[34

[465]	training's binary_logloss: 0.241726	valid_1's binary_logloss: 0.264729
[466]	training's binary_logloss: 0.241698	valid_1's binary_logloss: 0.26473
[467]	training's binary_logloss: 0.241666	valid_1's binary_logloss: 0.264726
[468]	training's binary_logloss: 0.241641	valid_1's binary_logloss: 0.26472
[469]	training's binary_logloss: 0.241615	valid_1's binary_logloss: 0.264723
[470]	training's binary_logloss: 0.241591	valid_1's binary_logloss: 0.264724
[471]	training's binary_logloss: 0.241561	valid_1's binary_logloss: 0.264724
[472]	training's binary_logloss: 0.241535	valid_1's binary_logloss: 0.264716
[473]	training's binary_logloss: 0.24151	valid_1's binary_logloss: 0.264711
[474]	training's binary_logloss: 0.241484	valid_1's binary_logloss: 0.264713
[475]	training's binary_logloss: 0.241457	valid_1's binary_logloss: 0.264716
[476]	training's binary_logloss: 0.24143	valid_1's binary_logloss: 0.264717
[477]	training's binary_logloss: 0.241395	valid_1's binary_logloss: 0.264717
[47

...

[521]	training's binary_logloss: 0.242017	valid_1's binary_logloss: 0.257965
[522]	training's binary_logloss: 0.241987	valid_1's binary_logloss: 0.257963
[523]	training's binary_logloss: 0.241966	valid_1's binary_logloss: 0.257963
[524]	training's binary_logloss: 0.241942	valid_1's binary_logloss: 0.257964
[525]	training's binary_logloss: 0.241919	valid_1's binary_logloss: 0.257965
[526]	training's binary_logloss: 0.241898	valid_1's binary_logloss: 0.257961
[527]	training's binary_logloss: 0.241874	valid_1's binary_logloss: 0.257959
[528]	training's binary_logloss: 0.241852	valid_1's binary_logloss: 0.257961
[529]	training's binary_logloss: 0.241828	valid_1's binary_logloss: 0.257962
[530]	training's binary_logloss: 0.241806	valid_1's binary_logloss: 0.257964
[531]	training's binary_logloss: 0.241785	valid_1's binary_logloss: 0.257969
[532]	training's binary_logloss: 0.241764	valid_1's binary_logloss: 0.257968
[533]	training's binary_logloss: 0.241731	valid_1's binary_logloss: 0.257974

[639]	training's binary_logloss: 0.239282	valid_1's binary_logloss: 0.25798
[640]	training's binary_logloss: 0.239256	valid_1's binary_logloss: 0.257984
[641]	training's binary_logloss: 0.239232	valid_1's binary_logloss: 0.257988
[642]	training's binary_logloss: 0.239205	valid_1's binary_logloss: 0.257987
[643]	training's binary_logloss: 0.23918	valid_1's binary_logloss: 0.257988
[644]	training's binary_logloss: 0.239161	valid_1's binary_logloss: 0.25799
[645]	training's binary_logloss: 0.239136	valid_1's binary_logloss: 0.257985
[646]	training's binary_logloss: 0.239114	valid_1's binary_logloss: 0.257987
[647]	training's binary_logloss: 0.239096	valid_1's binary_logloss: 0.25799
[648]	training's binary_logloss: 0.239078	valid_1's binary_logloss: 0.257991
[649]	training's binary_logloss: 0.239054	valid_1's binary_logloss: 0.25799
[650]	training's binary_logloss: 0.239034	valid_1's binary_logloss: 0.257985
[651]	training's binary_logloss: 0.23901	valid_1's binary_logloss: 0.257981
[652]

[753]	training's binary_logloss: 0.236911	valid_1's binary_logloss: 0.258023
[754]	training's binary_logloss: 0.236895	valid_1's binary_logloss: 0.258022
[755]	training's binary_logloss: 0.236876	valid_1's binary_logloss: 0.258022
[756]	training's binary_logloss: 0.236856	valid_1's binary_logloss: 0.258025
[757]	training's binary_logloss: 0.236835	valid_1's binary_logloss: 0.258026
[758]	training's binary_logloss: 0.236817	valid_1's binary_logloss: 0.258022
[759]	training's binary_logloss: 0.236795	valid_1's binary_logloss: 0.258022
[760]	training's binary_logloss: 0.236771	valid_1's binary_logloss: 0.258022
[761]	training's binary_logloss: 0.236755	valid_1's binary_logloss: 0.258023
[762]	training's binary_logloss: 0.236737	valid_1's binary_logloss: 0.258022
[763]	training's binary_logloss: 0.236715	valid_1's binary_logloss: 0.25802
[764]	training's binary_logloss: 0.236699	valid_1's binary_logloss: 0.258019
[765]	training's binary_logloss: 0.236679	valid_1's binary_logloss: 0.258021


[867]	training's binary_logloss: 0.234602	valid_1's binary_logloss: 0.258089
[868]	training's binary_logloss: 0.234578	valid_1's binary_logloss: 0.25809
[869]	training's binary_logloss: 0.234556	valid_1's binary_logloss: 0.25809
[870]	training's binary_logloss: 0.234532	valid_1's binary_logloss: 0.258088
[871]	training's binary_logloss: 0.234515	valid_1's binary_logloss: 0.258085
[872]	training's binary_logloss: 0.234493	valid_1's binary_logloss: 0.258084
[873]	training's binary_logloss: 0.234478	valid_1's binary_logloss: 0.258084
[874]	training's binary_logloss: 0.234464	valid_1's binary_logloss: 0.258084
[875]	training's binary_logloss: 0.234448	valid_1's binary_logloss: 0.258085
[876]	training's binary_logloss: 0.234433	valid_1's binary_logloss: 0.258087
[877]	training's binary_logloss: 0.23441	valid_1's binary_logloss: 0.258084
[878]	training's binary_logloss: 0.234392	valid_1's binary_logloss: 0.258085
[879]	training's binary_logloss: 0.234376	valid_1's binary_logloss: 0.258084
[8

[983]	training's binary_logloss: 0.23231	valid_1's binary_logloss: 0.258127
[984]	training's binary_logloss: 0.232292	valid_1's binary_logloss: 0.258125
[985]	training's binary_logloss: 0.23227	valid_1's binary_logloss: 0.258123
[986]	training's binary_logloss: 0.232256	valid_1's binary_logloss: 0.258128
[987]	training's binary_logloss: 0.23223	valid_1's binary_logloss: 0.258128
[988]	training's binary_logloss: 0.232205	valid_1's binary_logloss: 0.258127
[989]	training's binary_logloss: 0.232189	valid_1's binary_logloss: 0.258127
[990]	training's binary_logloss: 0.232167	valid_1's binary_logloss: 0.258128
[991]	training's binary_logloss: 0.23214	valid_1's binary_logloss: 0.258128
[992]	training's binary_logloss: 0.232125	valid_1's binary_logloss: 0.258125
[993]	training's binary_logloss: 0.2321	valid_1's binary_logloss: 0.258122
[994]	training's binary_logloss: 0.232082	valid_1's binary_logloss: 0.258124
[995]	training's binary_logloss: 0.232066	valid_1's binary_logloss: 0.258126
[996]

In [ ]:
# LightGBM
# Score by each fold: [0.70711, 0.71662, 0.71616, 0.70991, 0.71354]

## Задание 2

#### Усреднить прогнозы с помощью арифмитического среднего, геометрического среднего и усреднить ранги, сделать выводы о качестве отдельных моделей и о качестве комбинации.

## Задание 3

#### Обучить CatBoost, получить OOF прогнозы и выполнить задание 1 для трех моделей.
Выполнить задание 2 для трех моделей.

In [46]:
def catboost_cross_validation(params, X, y, cv, categorical = None):
    """
    Кросс-валидация для модели catbooost.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признако для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    cv: KFold or StratifiedKFold generator.
        Объект KFold / StratifiedKFold для определения
        стратегии кросс-валидации модели.

    categorical: str, optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    estimators: list
        Список с объектами обученной модели.

    oof_preds: np.array
        Вектор OOF-прогнозов.

    """
    estimators, folds_scores = [], []
    oof_preds = np.zeros(X.shape[0])

    print(f"{time.ctime()}, Cross-Validation, {X.shape[0]} rows, {X.shape[1]} cols")
    X[categorical] = X[categorical].astype(str)

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):

        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model = cb.CatBoostClassifier(**params)
        model.fit(
            x_train, y_train, categorical,
            eval_set=[(x_train, y_train), (x_valid, y_valid)]
        )
        oof_preds[valid_idx] = model.predict_proba(x_valid)[:, 1]
        score = roc_auc_score(y_valid, oof_preds[valid_idx])
        print(f"Fold {fold+1}, Valid score = {round(score, 5)}")
        folds_scores.append(round(score, 5))
        estimators.append(model)

    print(f"Score by each fold: {folds_scores}")
    print("="*65)
    return estimators, oof_preds

In [47]:
cb_params = {
    "n_estimators": 2000,
    "learning_rate": 0.01,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 0,
    "max_depth": 6,
    "l2_leaf_reg": 10,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 42
}

cv = KFold(n_splits=5, random_state=1234123, shuffle=True)

estimators, oof_preds = catboost_cross_validation(
    params=cb_params, X=X, y=y, cv=cv, categorical=cat_feats
)

Tue Sep 29 16:03:08 2020, Cross-Validation, 110093 rows, 24 cols
Fold 1, Valid score = 0.70849
Fold 2, Valid score = 0.71487
Fold 3, Valid score = 0.71383
Fold 4, Valid score = 0.70986
Fold 5, Valid score = 0.71806
Score by each fold: [0.70849, 0.71487, 0.71383, 0.70986, 0.71806]


In [48]:
# CatBoost
# Score by each fold: [0.70849, 0.71487, 0.71383, 0.70986, 0.71806]

(опция) Объединить OOF-прогнозы для трех моделей и обучить алгоритм Логистической регрессии (и любой другой, на ваше усмотрение). Сделать выводы о достигаемом качестве, сравнить достигаемое качество с качеством отдельных моделей и моделей, полученных в п.2 и п.4.

(опция) Обучить алгоритмRandomForest (желательно подтюнить параметры) и добавить к построенным ранее моделям. Выполнить задание 5.